In [1]:
from __future__ import print_function

from numba import jit
import numpy as np
from sklearn.utils.linear_assignment_ import linear_assignment
from filterpy.kalman import KalmanFilter


In [4]:
# 计算IOU
def iou(bb_test,bb_gt):
    """
    计算两个边界框(bounding box)的交并比(iou) ,输入格式[x1,y1,x2,y2]
    """
    xx1 = np.maximum(bb_test[0],bb_gt[0])
    yy1 = np.maximum(bb_test[1],bb_gt[1])
    xx2 = np.minimum(bb_test[2],bb_gt[2])
    yy2 = np.minimum(bb_test[3],bb_gt[3])
    w = np.maximum(0.,xx2-xx1)
    h = np.maximum(0.,yy2-yy1)
    wh = w*h
    o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1])+(bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1])-wh)
    return o

In [12]:
# 转换边界框表示[x,y,s,r]
def convert_bbox_to_z(bbox):
    """(x,y)为中心坐标，s为面积，r为宽高比"""
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w/2.
    y = bbox[1] + h/2.
    s = w * h
    r = w/float(h)
    return np.array([x,y,s,r]).reshape((4,1))

In [15]:
# [x,y,s,r]=>[x1,y1,x2,y2]
def convert_x_to_bbox(x,score=None):
    w = np.sqrt(x[2]*x[3])
    h = x[2]/w
    if(score==None):
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    else:
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

In [51]:
x=[1,2,3,4]
y=convert_bbox_to_z(x)
print(y)
z=convert_x_to_bbox(y)
print(z)
score = np.array([1.])
z=convert_x_to_bbox(y,score)
print(z)


[[2.]
 [3.]
 [4.]
 [1.]]
[[1. 2. 3. 4.]]
[[1. 2. 3. 4. 1.]]


In [29]:
# 单跟踪对象状态估计类，表示为bbox边界框
class KalmanBoxTracker(object):
    count = 0
    def __init__(self,bbox):
        """使用初始边界框初始化一个跟踪器"""
        # 定义匀速模型
        self.kf = KalmanFilter(dim_x=7,dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])
        
        self.kf.R[2:,2:] *=10.
        self.kf.P[4:,4:] *=1000.  # 对不可观测的初始速度给一个高的不确定性
        self.kf.P *=10.
        self.kf.Q[-1,-1] *=0.01
        self.kf.Q[4:,4:] *=0.01
        
        self.kf.x[:4] = convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0
        
    def update(self,bbox):
        """根据观测到的bbox更新状态向量"""
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(convert_bbox_to_z(bbox))
        
    def predict(self):
        """前进状态向量,并返回预测的边界框估计"""
        if((self.kf.x[6]+self.kf.x[2])<=0):
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age +=1
        if(self.time_since_update>=0):
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(convert_x_to_bbox(self.kf.x))
        return self.history[-1]
    
    def get_state(self):
        """返回当前的边界框估计"""
        return convert_x_to_bbox(self.kf.x)
    
    

In [54]:
def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
    """
    将检测值分配到跟踪对象（表示为bbox）
    返回3个列表：已匹配、未匹配检测值，未匹配跟踪器
    """
    if(len(trackers)==0) or (len(detections)==0):
        return np.empty((0,2),dtype=int),np.arange(len(detections)),np.empty((0,5),dtype=int)
    iou_matriix = np.zeros((len(detections),len(trackers)),dtype=np.float32)
    # 形成iou矩阵
    for d,det in enumerate(detections):
        for t,trk in enumerate(trackers):
            iou_matriix[d,t] = iou(det,trk)
    # 数据关联
    matched_indices = linear_assignment(-iou_matriix)  # 匈牙利算法   返回n个 1*2数组 输出行列序号
    
    unmatched_detection = []
    for d,det in enumerate(detections):
        if(d not in matched_indices[:,0]):
            unmatched_detection.append(d)
    unmatched_trackers = []
    for t,trk in enumerate(trackers):
        if(t not in matched_indices[:,1]):
            unmatched_trackers.append(t)
            
    # 去掉低iou的匹配项
    matches = []
    for m in matched_indices:
        if(iou_matriix[m[0],m[1]]<iou_threshold):
            unmatched_detection.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0)    # 数组拼接
        
    return matches,np.array(unmatched_detection),np.array(unmatched_trackers)
    
    

In [ ]:
class Sort(object)；